In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import requests

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
def player_cleaning(player):
    player = player.split("\n")
    player = [i.strip() for i in player]
    player = [i for i in player if i] 
    return player

def nba_stats(url, csv):
    ##########
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)

    driver.find_element_by_xpath("//body/main[1]/div[1]/div[1]/div[2]/div[1]/div[1]/nba-stat-table[1]/div[1]/div[1]/div[1]/select[1]").click()
    driver.find_element_by_xpath("//body/main[1]/div[1]/div[1]/div[2]/div[1]/div[1]/nba-stat-table[1]/div[1]/div[1]/div[1]/select[1]/option[1]").click()

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    driver.quit()

    ##########
    headers = [th.getText() for th in soup.find('div', {'class':'nba-stat-table__overflow'}).findAll('th')]
    #headers = [th.getText().strip() for th in soup.find('div', {'class':'nba-stat-table__overflow'}).findAll('th')]
    headers = [i for i in headers if "RANK" not in i]
    headers = [s.replace("\n", "") for s in headers]

    ##########
    player_rows = [tr.getText().strip() for tr in soup.tbody.findAll('tr')]
    player_rows = [player_cleaning(player) for player in player_rows]
    
    ##########
    df = pd.DataFrame(player_rows, columns = headers)
    df.to_csv(csv, index=False)
    
    return df

In [5]:
nba_stats("https://www.nba.com/stats/leaders/", "traditional22.csv").head(10)

[WDM] - Current google-chrome version is 99.0.4844
[WDM] - Get LATEST driver version for 99.0.4844
[WDM] - Driver [/Users/jeremydumalig/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


,#,Player,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF
0,1,LeBron James,56,37.2,30.3,11.4,21.8,52.4,2.9,8.0,35.9,4.5,6.0,75.6,1.1,7.1,8.2,6.2,1.3,1.1,3.5,31.7
1,2,Joel Embiid,65,33.6,30.2,9.6,19.5,49.3,1.4,3.7,36.8,9.6,11.8,81.7,2.1,9.4,11.6,4.2,1.1,1.5,3.1,33.5
2,3,Giannis Antetokounmpo,64,33.0,30.1,10.3,18.7,55.4,1.1,3.6,29.6,8.3,11.5,72.1,2.0,9.6,11.6,5.8,1.1,1.4,3.3,35.1
3,4,Luka Doncic,62,35.6,28.3,9.9,21.7,45.6,3.1,8.7,35.3,5.5,7.4,74.4,0.9,8.3,9.1,8.7,1.2,0.5,4.5,29.7
4,5,Trae Young,72,34.8,28.3,9.3,20.3,45.9,3.1,8.0,38.3,6.6,7.3,90.3,0.6,3.1,3.8,9.6,1.0,0.1,4.0,27.1
5,6,DeMar DeRozan,73,36.2,28.0,10.2,20.3,50.3,0.7,1.9,35.5,6.9,7.9,87.7,0.7,4.5,5.3,5.0,0.9,0.3,2.3,26.1
6,7,Ja Morant,56,33.3,27.6,10.2,20.8,49.3,1.5,4.5,34.0,5.6,7.3,76.2,1.3,4.4,5.7,6.7,1.2,0.4,3.4,25.8
7,8,Jayson Tatum,74,36.1,27.0,9.4,20.7,45.2,3.0,8.6,35.1,5.3,6.2,85.4,1.1,6.8,8.0,4.4,1.0,0.6,2.8,25.9
8,9,Nikola Jokic,72,33.3,26.8,10.1,17.4,58.3,1.3,3.9,33.9,5.2,6.4,81.1,2.8,10.9,13.7,8.0,1.4,0.9,3.8,38.5
9,10,Devin Booker,66,34.5,26.6,9.7,20.8,46.4,2.7,7.0,37.7,4.7,5.4,86.5,0.7,4.3,5.0,4.9,1.2,0.4,2.4,23.8


In [ ]:
##########
headers = []
player_rows = []
    
def parser(url):
    ##########
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    ##########
    headers.append( [td.getText() for td in soup.findAll('tr', {"class":"colhead"})[0]] )

    ##########
    player_rows.append( [[td.getText() for td in i] for i in soup.tbody.findAll('tr')[2:]] )

    ##########
    driver.quit()
    
def insider(url, page_count):
    global df
    url = str(url + "/_/league/nba/page/")
    
    ##########
    for page_index in range(1, page_count + 1):
        parser(url + str(page_index))
    
    ##########
    players = []
    [[players.append(i) for i in item] for item in player_rows]

    ##########
    df = pd.DataFrame(players, columns = headers[0])
    
    df['PLAYER'], df['POS'] = df['PLAYER'].str.split(',', 1).str
    
    new_headers = list(df.columns)
    new_headers.remove("POS")
    new_headers.insert(2, "POS")
    df = df[new_headers]
    
    df = df[df['PLAYER'] != 'PLAYER']
    
    display(df.head(10))
    
    #df.to_csv('insider.csv', index=False)

In [ ]:
def entries1(tr_tag):
    rank = tr_tag.find('td').getText()
    name = tr_tag.findAll('td')[1].find('a').getText()
    team = tr_tag.findAll('td')[1].find('span').getText()
    return [rank, name, team]

def espn(url):
    ##########
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)

    show_more_xpath = "//a[@class='AnchorLink loadMore__link']"

    switch = True
    while switch == True:
        try:
            driver.find_element_by_xpath(show_more_xpath).click()
        except:
            switch = False

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    driver.quit()

    ##########
    headers1 = [i.getText() for i in soup.findAll('tr', {"class":"Table__sub-header Table__TR Table__even"})[0].findAll('th')]
    headers1.insert(2, "Team")
    headers2 = [i.getText() for i in soup.findAll('tr', {"class":"Table__sub-header Table__TR Table__even"})[1].findAll('th')]

    headers = headers1 + headers2

    ##########
    player_rows1 = [entries1(i) for i in soup.tbody.findAll('tr')]
    player_rows2 = [[td.getText() for td in i] for i in soup.findAll('tbody', {"class":"Table__TBODY"})[1].findAll('tr')]

    player_rows = [a + b for a, b in zip(player_rows1, player_rows2)]

    ##########
    df = pd.DataFrame(player_rows, columns = headers)
    
    return df
    
    #df.to_csv('espn.csv', index=False)

In [ ]:
def basketball_reference(url):
    ##########
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    driver.quit()

    ##########
    headers = [th.getText() for th in soup.findAll('thead', limit=2)[0].findAll('th')]

    ##########
    player_rows1 = [[i.find('th').text] for i in soup.findAll('tr', {"class":"full_table"})]
    player_rows2 = [[td.getText()for td in i.findAll('td')] for i in soup.findAll('tr', {"class":"full_table"})]
    player_rows = [a + b for a, b in zip(player_rows1, player_rows2)]

    ##########
    df = pd.DataFrame(player_rows, columns = headers)
    
    return df
    
    #df.to_csv('basketball_reference.csv', index=False)

In [ ]:
def wnba(url):
    ##########
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)

    driver.find_element_by_xpath("//body//nba-stat-table//div[1]//div[1]//div[1]//select[1]//option[2]").click()
    driver.find_element_by_xpath("//body//nba-stat-table//div[1]//div[1]//div[1]//select[1]//option[1]").click()

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    driver.quit()

    ##########
    player_rows = [[td.getText().strip() for td in i.findAll('td')] for i in soup.tbody.findAll('tr')]

    ##########
    headers = [th.getText().strip() for th in soup.thead.findAll('th')]
    headers = headers[0:len(player_rows[0])]

    ##########
    df = pd.DataFrame(player_rows, columns = headers)
    
    return df
    
    #df.to_csv('wnba.csv', index=False)

In [ ]:
#nba_stats("https://www.nba.com/stats/leaders/", "traditional22.csv")
#espn("https://www.espn.com/nba/stats/player")
#basketball_reference("https://www.basketball-reference.com/leagues/NBA_2022_per_game.html#per_game_stats::pts_per_ghttps://www.basketball-reference.com/leagues/NBA_2022_per_game.html#per_game_stats::pts_per_g")
#wnba("https://stats.wnba.com/players/traditional/?DateFrom=&DateTo=&PerMode=Totals&sort=PTS&dir=-1")